# Example of `InferenceData` schema in PyStan
The description of the `InferenceData` structure can be found in [GitHub](https://github.com/arviz-devs/arviz/blob/master/schema.md).

In [1]:
import arviz as az
import pystan
import pandas as pd
import numpy as np
import xarray
xarray.set_options(display_style="html");

In [2]:
#read data
data = pd.read_csv("linear_regression_data.csv", index_col=0)
time_since_joined = data.time.values
slack_comments = data.comments.values
names = data.index.values
N = len(names)
data

,comments,time
Alice,7500,4.5
Bob,10100,6.0
Cole,18600,7.0
Danielle,25200,12.0
Erika,27500,18.0


In [3]:
linreg_prior_code = """
data {
  int<lower=0> N;
  real time_since_joined[N];
}

generated quantities {
    real b0;
    real b1;
    real log_sigma;
    real<lower=0> sigma;
    vector[N] slack_comments_hat;
    
    b0 = normal_rng(0,10);
    b1 = normal_rng(0,10);
    sigma = abs(normal_rng(0,30));
    log_sigma = log(sigma);
    for (i in 1:N) {
        slack_comments_hat[i] = normal_rng(b0 + b1 * time_since_joined[i], sigma);
    }
}
"""
sm_prior = pystan.StanModel(model_code=linreg_prior_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_b2d530337b2d60d5322cb10cbc01e530 NOW.


In [4]:
linreg_prior_data_dict = {"N": N, "time_since_joined": time_since_joined}
prior = sm_prior.sampling(data=linreg_prior_data_dict, iter=200, chains=1, algorithm='Fixed_param', warmup=0)

In [5]:
linreg_code = """
data {
  int<lower=0> N;
  real time_since_joined[N];
  real slack_comments[N];
}

parameters {
  real b0;
  real b1;
  real log_sigma;
}

transformed parameters {
  real<lower=0> sigma;
  sigma = exp(log_sigma);
}

model {
  b0 ~ normal(0,10);
  b1 ~ normal(0,10);
  sigma ~ normal(0,30);
  for (i in 1:N) {
    slack_comments[i] ~ normal(b0 + b1 * time_since_joined[i], sigma);
  }
  
}

generated quantities {
    vector[N] log_lik;
    vector[N] slack_comments_hat;
    for (i in 1:N) {
        log_lik[i] = normal_lpdf(slack_comments[i] | b0 + b1 * time_since_joined[i], sigma);
        slack_comments_hat[i] = normal_rng(b0 + b1 * time_since_joined[i], sigma);
    }
}
"""
sm = pystan.StanModel(model_code=linreg_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_13939a7ccc2875c5ec3f9d016bdf59f1 NOW.


In [6]:
linreg_data_dict = {"N": N, "slack_comments": slack_comments, "time_since_joined": time_since_joined}
posterior = sm.sampling(data=linreg_data_dict, iter=200, chains=4)

In [7]:
idata_stan = az.from_pystan(
    posterior=posterior,
    prior=prior,
    posterior_predictive="slack_comments_hat",
    prior_predictive="slack_comments_hat",
    observed_data=["slack_comments"],
    constant_data=["time_since_joined"],
    log_likelihood="log_lik",
    coords={"developer": names},
    dims={
        "slack_comments": ["developer"],
        "log_lik": ["developer"],
        "slack_comments_hat": ["developer"],
        "time_since_joined": ["developer"],
    }
)

In [8]:
idata_stan

Inference data with groups:
	> posterior
	> sample_stats
	> posterior_predictive
	> prior
	> sample_stats_prior
	> prior_predictive
	> observed_data
	> constant_data

In this example, each variable has as dimension a combination of the following 3: `chain`, `draw` and `developer`. Moreover, each dimension has specific coordinate values. In the case of `chain` and `draw` it is an integer identifier starting at `0`; in the case of `developer` dimension, its coordinate values are the following strings: `["Alice", "Bob", "Cole", "Danielle", "Erika"]`.

In [9]:
idata_stan.posterior

<xarray.Dataset>
Dimensions:    (chain: 4, draw: 100)
Coordinates:
  * chain      (chain) int64 0 1 2 3
  * draw       (draw) int64 0 1 2 3 4 5 6 7 8 9 ... 91 92 93 94 95 96 97 98 99
Data variables:
    b0         (chain, draw) float64 30.07 30.06 26.52 ... 12.01 -9.397 2.975
    b1         (chain, draw) float64 75.13 75.13 73.66 ... 70.07 79.98 88.35
    log_sigma  (chain, draw) float64 7.035 7.009 7.03 ... 6.997 7.028 7.003
    sigma      (chain, draw) float64 1.135e+03 1.107e+03 ... 1.128e+03 1.1e+03
Attributes:
    created_at:                 2019-11-10T23:49:36.689811
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [10]:
idata_stan.sample_stats

<xarray.Dataset>
Dimensions:         (chain: 4, developer: 5, draw: 100)
Coordinates:
  * chain           (chain) int64 0 1 2 3
  * draw            (draw) int64 0 1 2 3 4 5 6 7 8 ... 92 93 94 95 96 97 98 99
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    accept_stat     (chain, draw) float64 0.8763 0.8703 0.9905 ... 0.8481 0.8889
    stepsize        (chain, draw) float64 0.01492 0.01492 ... 0.01627 0.01627
    treedepth       (chain, draw) int64 10 2 10 10 10 10 10 ... 10 9 7 8 8 10 10
    n_leapfrog      (chain, draw) int64 1023 3 1023 1023 ... 499 499 1023 1023
    diverging       (chain, draw) bool False False False ... False False False
    energy          (chain, draw) float64 1.46e+03 1.46e+03 ... 1.458e+03
    log_likelihood  (chain, draw, developer) float64 -27.68 -43.84 ... -285.2
    lp              (chain, draw) float64 -1.459e+03 -1.459e+03 ... -1.457e+03
Attributes:
    created_at:                 2019-11-10T23:49:36.693939
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [11]:
idata_stan.posterior_predictive

<xarray.Dataset>
Dimensions:             (chain: 4, developer: 5, draw: 100)
Coordinates:
  * chain               (chain) int64 0 1 2 3
  * draw                (draw) int64 0 1 2 3 4 5 6 7 ... 92 93 94 95 96 97 98 99
  * developer           (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    slack_comments_hat  (chain, draw, developer) float64 169.6 ... 2.244e+03
Attributes:
    created_at:                 2019-11-10T23:49:36.696524
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [12]:
idata_stan.observed_data

<xarray.Dataset>
Dimensions:         (developer: 5)
Coordinates:
  * developer       (developer) object 'Alice' 'Bob' 'Cole' 'Danielle' 'Erika'
Data variables:
    slack_comments  (developer) int64 7500 10100 18600 25200 27500
Attributes:
    created_at:                 2019-11-10T23:49:36.684431
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [13]:
idata_stan.constant_data

<xarray.Dataset>
Dimensions:            (developer: 5)
Coordinates:
  * developer          (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    time_since_joined  (developer) float64 4.5 6.0 7.0 12.0 18.0
Attributes:
    created_at:                 2019-11-10T23:49:36.687422
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [14]:
idata_stan.prior

<xarray.Dataset>
Dimensions:    (chain: 1, draw: 200)
Coordinates:
  * chain      (chain) int64 0
  * draw       (draw) int64 0 1 2 3 4 5 6 7 ... 192 193 194 195 196 197 198 199
Data variables:
    b0         (chain, draw) float64 1.915 -7.531 -13.56 ... 15.67 5.08 13.27
    b1         (chain, draw) float64 -7.451 -4.739 -3.937 ... 10.36 -2.903 13.49
    log_sigma  (chain, draw) float64 1.069 3.28 3.924 ... 3.166 3.393 1.744
    sigma      (chain, draw) float64 2.912 26.57 50.61 ... 23.71 29.77 5.72
Attributes:
    created_at:                 2019-11-10T23:49:36.697872
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [15]:
idata_stan.sample_stats_prior

<xarray.Dataset>
Dimensions:      (chain: 1, draw: 200)
Coordinates:
  * chain        (chain) int64 0
  * draw         (draw) int64 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
Data variables:
    accept_stat  (chain, draw) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    lp           (chain, draw) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    created_at:                 2019-11-10T23:49:36.699521
    inference_library:          pystan
    inference_library_version:  2.19.1.1

In [16]:
idata_stan.prior_predictive

<xarray.Dataset>
Dimensions:             (chain: 1, developer: 5, draw: 200)
Coordinates:
  * chain               (chain) int64 0
  * draw                (draw) int64 0 1 2 3 4 5 6 ... 194 195 196 197 198 199
  * developer           (developer) object 'Alice' 'Bob' ... 'Danielle' 'Erika'
Data variables:
    slack_comments_hat  (chain, draw, developer) float64 -30.78 -43.78 ... 256.5
Attributes:
    created_at:                 2019-11-10T23:49:36.700828
    inference_library:          pystan
    inference_library_version:  2.19.1.1